In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pylorentz import Momentum4

In [1]:
def gen_Files():
    
    #FORMAT:
    # mcWeight, SumWeights, sf_Pileup, sf_Photon, sf_Btag, sf_Trigger, 
    # Gam1(pt,eta,phi,E, trigM, Tight, ptCone, etaCone),Gam2(...), lep_n, trigP, jet_n, jet1(...,Btag),...

    #Input files to read, scale factor files to write, and data files to write
    in_files = ['ggHout.csv', 'dataAout.csv', 'dataBout.csv', 'dataCout.csv','dataDout.csv', 'yyjj_p1.csv']
    sf_files = ['ggH_sf.csv', 'dataA_sf.csv', 'dataB_sf.csv', 'dataC_sf.csv', 'dataD_sf.csv']
    data_files = ['ggH.csv','dataA.csv','dataB.csv','dataC.csv','dataD.csv']
    
    #Due to ragged CSV files, must find the file with the longest row
    #Then column names can be generated and passed to pandas
    #when opening the file
    max_lens = []
    for i in range(len(in_files)):

        max_row = len(max(open('./../CSVfiles/{}'.format(in_files[i]), 'r'),key=len).split(',')) -1
        max_lens.append(max_row)

    biggest_row = max(max_lens) #This is 100, so make col names 100 long
    col_names = ['mcweight', 'sumweights', 'sf_pileup', 'sf_photon', 'sf_btag', 'sf_trigger', 'g1_pt','g1_eta',
                 'g1_phi','g1_E','g1_trigm','g1_tight','g1_ptcone','g1_etcone','g2_pt','g2_eta',
                 'g2_phi','g2_E','g2_trigm','g2_tight','g2_ptcone','g2_etcone', 'lep_n', 'trigP', 'jet_n',
                 'j1_pt','j1_eta','j1_phi','j1_E','j1_MV2C10','j2_pt','j2_eta','j2_phi','j2_E','j2_MV2C10',
                 'j3_pt','j3_eta','j3_phi','j3_E','j3_MV2C10','j4_pt','j4_eta','j4_phi','j4_E','j4_MV2C10',
                 'j5_pt','j5_eta','j5_phi','j5_E','j5_MV2C10','j6_pt','j6_eta','j6_phi','j6_E','j6_MV2C10',
                 'j7_pt','j7_eta','j7_phi','j7_E','j7_MV2C10','j8_pt','j8_eta','j8_phi','j8_E','j8_MV2C10',
                 'j9_pt','j9_eta','j9_phi','j9_E','j9_MV2C10','j10_pt','j10_eta','j10_phi','j10_E','j10_MV2C10',
                 'j11_pt','j11_eta','j11_phi','j11_E','j11_MV2C10','j12_pt','j12_eta','j12_phi','j12_E','j12_MV2C10',
                 'j13_pt','j13_eta','j13_phi','j13_E','j13_MV2C10','j14_pt','j14_eta','j14_phi','j14_E','j14_MV2C10',
                 'j15_pt','j15_eta','j15_phi','j15_E','j15_MV2C10']
    
    #Parse through each file, process it, then write to the new files
    for i in range(len(in_files)):
        
        #For all but the MC background file (becuase this file has a unique structure)
        if (in_files[i] != 'yyjj_p1.csv'):

            #Read CSV
            df = pd.read_csv('./../CSVfiles/{}'.format(in_files[i]), names = col_names)

            #Replace NaN values with integer
            df.fillna(-10**6, inplace = True)

            #Filter out entries with > 0 leptons, then get rid of the column
            indexes = df.loc[df['lep_n'] > 0].index
            df.drop(indexes,inplace = True)
            df.drop(['lep_n', 'jet_n',
                 'j3_pt','j3_eta','j3_phi','j3_E','j3_MV2C10','j4_pt','j4_eta','j4_phi','j4_E','j4_MV2C10',
                 'j5_pt','j5_eta','j5_phi','j5_E','j5_MV2C10','j6_pt','j6_eta','j6_phi','j6_E','j6_MV2C10',
                 'j7_pt','j7_eta','j7_phi','j7_E','j7_MV2C10','j8_pt','j8_eta','j8_phi','j8_E','j8_MV2C10',
                 'j9_pt','j9_eta','j9_phi','j9_E','j9_MV2C10','j10_pt','j10_eta','j10_phi','j10_E','j10_MV2C10',
                 'j11_pt','j11_eta','j11_phi','j11_E','j11_MV2C10','j12_pt','j12_eta','j12_phi','j12_E','j12_MV2C10',
                 'j13_pt','j13_eta','j13_phi','j13_E','j13_MV2C10','j14_pt','j14_eta','j14_phi','j14_E','j14_MV2C10',
                 'j15_pt','j15_eta','j15_phi','j15_E','j15_MV2C10'], axis=1, inplace = True)
                
            #Use PyLorentz to calculate the quantities associated with the
            #parent of the two photons
            inv_masses, trans_momenta, energies, etas, phis = parent_Quantities(df[['g1_pt', 'g1_eta', 'g1_phi', 'g1_E',
                                                                                    'g2_pt', 'g2_eta', 'g2_phi', 'g2_E']].values)


            #Calculate photon separation angle (eq: sqrt(eta1-eta2)^2+(phi1-phi2)^2))

            del_r = np.sqrt((df['g1_eta']-df['g2_eta'])**2 + (df['g1_phi']-df['g2_phi'])**2)

            #Add the new columns to the dataframe
            df.insert(loc=0, column='photon_sep', value=del_r)
            df.insert(loc=0, column='p_E', value=energies)
            df.insert(loc=0, column='p_phi', value=phis)
            df.insert(loc=0, column='p_eta', value=etas)
            df.insert(loc=0, column='p_pt', value=trans_momenta)
            df.insert(loc=0, column='p_mass', value=inv_masses)
            
            #Can remove rows of data set if at least 1 photon: is not tight, 
            #or has ptCone/pt > 0.05, or has etaCone/eta > 0.065, or mass outside 90 - 170 GeV range
            pt1Ratio = df['g1_ptcone']/df['g1_pt']
            pt2Ratio = df['g2_ptcone']/df['g2_pt']
            et1Ratio = df['g1_etcone']/df['g1_pt']
            et2Ratio = df['g2_etcone']/df['g2_pt']

            remove_tight = df[(df['g1_tight'] == 0) | (df['g2_tight'] == 0)].index
            print('Tight condition: %.2f'%(len(remove_tight)/len(df)))
            remove_ptratio = df[(pt1Ratio > 0.05) | (pt2Ratio > 0.05)].index
            print('Ptratio condition: %.2f'%(len(remove_ptratio)/len(df)))
            remove_etratio = df[(et1Ratio > 0.065) | (et2Ratio > 0.065)].index
            print('Etratio condition: %.2f'%(len(remove_etratio)/len(df)))
            remove_mass = df[(df['p_mass'] < 105) | (df['p_mass'] > 165)].index
            print('Mass condition: %.2f'%(len(remove_mass)/len(df)))
            remove_mass_ratio = df[(df['g1_pt']/df['p_mass']) > 0.35 | (df['g2_pt']/df['p_mass']) > 0.25].index
            print('Pt/Mass condition: %.2f'%(len(remove_mass)/len(df)))
            
            del remove_tight,remove_ptratio,remove_etratio,remove_mass
            
            inds_to_remove = df[((df['g1_tight'] == 0) | (df['g2_tight'] == 0)) | ((pt1Ratio > 0.05) 
                            | (pt2Ratio > 0.05)) | ((et1Ratio > 0.065) | (et2Ratio > 0.065)) 
                            | ((df['p_mass'] < 105) | (df['p_mass'] > 165))
                            | ((df['g1_pt']/df['p_mass']) > 0.35 | (df['g2_pt']/df['p_mass']) > 0.25)].index
            print('Overall: %.2f'%(len(inds_to_remove)/len(df)))
            df.drop(inds_to_remove, axis = 0, inplace = True)
            
            #Make masses in GeV
            df['p_mass'] = df['p_mass']/1000
            
            #Generate a label column for ggH data
            if (in_files[i] == 'ggHout.csv'):
                labels = np.ones(len(df))
                df.insert(loc=0, column='label', value=labels)

            #Isolate scale factor data, then remove this to isolate particle data
            sf_df = df[['mcweight', 'sumweights', 'sf_pileup', 'sf_photon', 'sf_btag', 'sf_trigger']]
            df.drop(['mcweight', 'sumweights', 'sf_pileup', 'sf_photon', 'sf_btag', 'sf_trigger'], axis=1, inplace = True)

            #Write scale factor data
            sf_df.to_csv("./../CSVfiles/{}".format(sf_files[i]), index = False, header = True)
            #Write all other data
            df.to_csv("./../CSVfiles/{}".format(data_files[i]), index = False, header = True)
                
            print('Done {}'.format(in_files[i]))
            
        elif (in_files[i] == 'yyjj_p1.csv'): 
            gen_ggjj()

In [1]:
#We need a different function for the ggjj dataset, as it has a differnet layout
def gen_ggjj():
    
    #FORMAT:
    # m_weight,y1_Pt ,y1_Eta ,y1_Phi , y1_E, m_y1_is_tight ,m_y1_ptcone20 ,
    #m_y1_topoetcone20,m_y1_is_isolated, y2.Pt ,y2.Eta ,y2.Phi , y2.E,
    #m_y2_is_tight ,m_y2_ptcone20 ,m_y2_topoetcone20, m_y2_is_isolated, 
    #myy.M, njet, jet.pt ,jet.eta  jet.phi ,jet.e ,is_btag_70
    
    #THROW OUT:
    #m_y1_ptcone20, m_y1_topoetcone20, m_y1_is_isolated, _is_btag_70 
    
    col_names = ['m_weight', 'g1_pt','g1_eta',
                 'g1_phi','g1_E','g1_tight','g1_ptcone','g1_etcone','m_y1_is_isolated','g2_pt','g2_eta',
                 'g2_phi','g2_E','g2_tight','g2_ptcone','g2_etcone','m_y2_is_isolated', 'p_mass', 'jet_n',
                 'j1_pt','j1_eta','j1_phi','j1_E','j1_MV2C10','j2_pt','j2_eta','j2_phi','j2_E','j2_MV2C10',
                 'j3_pt','j3_eta','j3_phi','j3_E','j3_MV2C10','j4_pt','j4_eta','j4_phi','j4_E','j4_MV2C10',
                 'j5_pt','j5_eta','j5_phi','j5_E','j5_MV2C10','j6_pt','j6_eta','j6_phi','j6_E','j6_MV2C10',
                 'j7_pt','j7_eta','j7_phi','j7_E','j7_MV2C10','j8_pt','j8_eta','j8_phi','j8_E','j8_MV2C10',
                 'j9_pt','j9_eta','j9_phi','j9_E','j9_MV2C10','j10_pt','j10_eta','j10_phi','j10_E','j10_MV2C10',
                 'j11_pt','j11_eta','j11_phi','j11_E','j11_MV2C10','j12_pt','j12_eta','j12_phi','j12_E','j12_MV2C10',
                 'j13_pt','j13_eta','j13_phi','j13_E','j13_MV2C10','j14_pt','j14_eta','j14_phi','j14_E','j14_MV2C10',
                 'j15_pt','j15_eta','j15_phi','j15_E','j15_MV2C10']

    #Read CSV
    df = pd.read_csv('./../CSVfiles/yyjj_p1.csv', names = col_names)

    #Replace NaN values with integer
    df.fillna(-10**6, inplace = True)
        
    #Get rid of columns with no dataABCD equivalent
    df.drop(['m_y1_is_isolated', 'm_y2_is_isolated', 'jet_n',
                 'j3_pt','j3_eta','j3_phi','j3_E','j3_MV2C10','j4_pt','j4_eta','j4_phi','j4_E','j4_MV2C10',
                 'j5_pt','j5_eta','j5_phi','j5_E','j5_MV2C10','j6_pt','j6_eta','j6_phi','j6_E','j6_MV2C10',
                 'j7_pt','j7_eta','j7_phi','j7_E','j7_MV2C10','j8_pt','j8_eta','j8_phi','j8_E','j8_MV2C10',
                 'j9_pt','j9_eta','j9_phi','j9_E','j9_MV2C10','j10_pt','j10_eta','j10_phi','j10_E','j10_MV2C10',
                 'j11_pt','j11_eta','j11_phi','j11_E','j11_MV2C10','j12_pt','j12_eta','j12_phi','j12_E','j12_MV2C10',
                 'j13_pt','j13_eta','j13_phi','j13_E','j13_MV2C10','j14_pt','j14_eta','j14_phi','j14_E','j14_MV2C10',
                 'j15_pt','j15_eta','j15_phi','j15_E','j15_MV2C10'], axis = 1, inplace = True)
    
    #Can remove rows of data set if at least 1 photon: is not tight, 
    #or has ptCone/pt > 0.05, or has etaCone/eta > 0.065, or mass outside 105 - 165 GeV range

    pt1Ratio = df['g1_ptcone']/df['g1_pt']
    pt2Ratio = df['g2_ptcone']/df['g2_pt']
    et1Ratio = df['g1_etcone']/df['g1_pt']
    et2Ratio = df['g2_etcone']/df['g2_pt']

    remove_tight = df[(df['g1_tight'] == 0) | (df['g2_tight'] == 0)].index
    print('Tight condition: %.2f'%(len(remove_tight)/len(df)))
    remove_ptratio = df[(pt1Ratio > 0.05) | (pt2Ratio > 0.05)].index
    print('Ptratio condition: %.2f'%(len(remove_ptratio)/len(df)))
    remove_etratio = df[(et1Ratio > 0.065) | (et2Ratio > 0.065)].index
    print('Etratio condition: %.2f'%(len(remove_etratio)/len(df)))
    remove_mass = df[(df['p_mass'] < 105) | (df['p_mass'] > 165)].index
    print('Mass condition: %.2f'%(len(remove_mass)/len(df)))
    remove_mass_ratio = df[(df['g1_pt']/df['p_mass']) > 0.35 | (df['g2_pt']/df['p_mass']) > 0.25].index
    print('Pt/Mass condition: %.2f'%(len(remove_mass)/len(df)))
            
    del remove_tight,remove_ptratio,remove_etratio,remove_mass
            
    inds_to_remove = df[((df['g1_tight'] == 0) | (df['g2_tight'] == 0)) | ((pt1Ratio > 0.05) 
                    | (pt2Ratio > 0.05)) | ((et1Ratio > 0.065) | (et2Ratio > 0.065)) 
                    | ((df['p_mass'] < 105) | (df['p_mass'] > 165))
                    | ((df['g1_pt']/df['p_mass']) > 0.35 | (df['g2_pt']/df['p_mass']) > 0.25)].index
    print('Overall: %.2f'%(len(inds_to_remove)/len(df)))
    df.drop(inds_to_remove, axis = 0, inplace = True)
    
    #Masses in GeV
    df['p_mass'] = df['p_mass']/1000
    
    #Calculate photon separation angle (eq: sqrt(eta1-eta2)^2+(phi1-phi2)^2))
    del_r = np.sqrt((df['g1_eta']-df['g2_eta'])**2 + (df['g1_phi']-df['g2_phi'])**2)
        
    #This is a data file of background events, hence
    #all labels will be 0
    truths = np.zeros(len(df))
    
    #Use PyLorentz to calculate the quantities associated with the
    #parent of the two photons
    inv_masses, trans_momenta, energies, etas, phis = parent_Quantities(df[['g1_pt', 'g1_eta', 'g1_phi', 'g1_E',
                                                                            'g2_pt', 'g2_eta', 'g2_phi', 'g2_E']].values)
            
    #Add the new columns to the dataframe
    #To run ML, we will need the outputted csv to have the same number of columns 
    #as the ggH and dataABCD files, so fill the duds with NaN replacement
    #***However, should drop such columns before training as it will
    #allow the algorithm to cheat***
    
    df.insert(loc=0, column='photon_sep', value=del_r)
    df.insert(loc=0, column='p_E', value=energies)
    df.insert(loc=0, column='p_phi', value=phis)
    df.insert(loc=0, column='p_eta', value=etas)
    df.insert(loc=0, column='p_pt', value=trans_momenta)
    df.insert(loc=0, column='label', value=truths)
    df.insert(loc=11, column='g1_trigm', value=-1000000)
    df.insert(loc=19, column='g2_trigm', value=-1000000)
    df.insert(loc=23, column='trigP', value=-1000000)
        
    #Move mass column to start of dataframe (mass is currently the 24th column)
    cols = df.columns.tolist()
    cols = [cols[0]] + [cols[24]] + cols[1:24] + cols[25:]
    df = df[cols]

    #Isolate scale factor data, then remove this to isolate particle data
    sf_df = df[['m_weight']]
    df.drop(['m_weight'], axis=1, inplace = True)
        
    sf_df.to_csv('./../CSVfiles/sf_yyjj_p1.csv', index = False, header = True)
    df.to_csv('./../CSVfiles/data_yyjj_p1.csv', index = False, header = True)
        
    print('Done ggjj')

In [4]:
#Calculates truth vales upon a threshold cut
def calc_Truths(y_pred, y_val, m, sf_val, threshold):

    true_pos = []
    false_pos = []
    true_neg = []
    false_neg = []

    for i in range(len(y_val)):

        if y_val[i] == 1 and y_pred[i] >= threshold:
            true_pos.append([m[i], sf_val[i]])
        elif y_val[i] == 0 and y_pred[i] >= threshold:
            false_pos.append([m[i], sf_val[i]])
        elif y_val[i] == 1 and y_pred[i] < threshold:
            false_neg.append([m[i], sf_val[i]])
        elif y_val[i] == 0 and y_pred[i] < threshold:
            true_neg.append([m[i], sf_val[i]])

    return true_pos, false_pos, true_neg, false_neg

In [1]:
def Sensitivity(true_pos, false_pos, sf_trim, SF_band):
    
    s, b, band_bkg = 0, 0, 0
    for i in range(len(true_pos)):
        if (true_pos[i][0] > 121) and (true_pos[i][0] < 129):
            s += true_pos[i][1]
    for i in range(len(false_pos)):
        if (false_pos[i][0] > 121) and (false_pos[i][0] < 129):
            b += false_pos[i][1]
        elif (false_pos[i][0] <= 121) or (false_pos[i][0] >= 129):
            band_bkg += false_pos[i][1]
    
    s = s*sf_trim[0]*(2.00419*(10**-3))
    b = b*sf_trim[1]*SF_band
    n = (s+b)
    print('%.2f '%s,' %.2f '%b,' %.2f '%band_bkg)
    sensitivity = np.sqrt(2*(n*np.log(n/b)+b-n))

    return sensitivity

In [6]:
#Used to turn a list into PyLorentz quantities
def lorentzify(lst):

    gamma_objects = []

    #Separate each photon
    each_gamma = np.split(lst, 2)

    #Change each gamma into a PyLorentz object
    for j in range(2):
        gamma_objects.append(Momentum4.e_eta_phi_pt(each_gamma[j][3],each_gamma[j][1], each_gamma[j][2], each_gamma[j][0]))

    return gamma_objects

In [7]:
#Use PyLorentz to calculate parent particle quantities
def parent_Quantities(lst):

    #Set memory placeholders for each list to avoid appends
    inv_masses = np.zeros(len(lst))
    trans_momenta = np.zeros(len(lst))
    energies = np.zeros(len(lst))
    etas = np.zeros(len(lst))
    phis = np.zeros(len(lst))

    for i in range(len(lst)):

        #Turn list into PyLorentz objects
        gammas = lorentzify(lst[i])
        parent = gammas[0] + gammas[1]

        #Calculate quantities
        inv_masses[i] = parent.m
        trans_momenta[i] = parent.p_t
        energies[i] = parent.e
        etas[i] = parent.eta
        phis[i] = parent.phi
    
    return inv_masses, trans_momenta, energies, etas, phis

In [3]:
def profile (m, ys, labels=None, bins=np.linspace(100,170,50,endpoint=True), ax=None):
    
    plt.rcParams.update({'font.size': 20})
    plt.figure(figsize=(12,6), dpi= 100)
    # Check(s)
    if isinstance(bins, int):
        bins = np.linspace(m.min(), m.max(), bins + 1, endpoint=True)
        pass

    if not isinstance(ys, list):
        ys = [ys]
        pass

    N = len(ys)
    centres = bins[:-1] + 0.5 * np.diff(bins)

    if labels is None:
        labels = [None for _ in range(N)]
    elif isinstance(labels, str):
        labels = [labels]
        pass

    assert len(labels) == N, "[profile] Number of observables ({}) and associated labels ({}) do not match.".format(N, len(labels))

    # Local background efficiency
    profiles = {ix: list() for ix in range(N)}
    means_NN  = list()
    means_ANN = list()
    for down, up in zip(bins[:-1], bins[1:]):
        msk = (m >= down) & (m < up)
        for ix, y in enumerate(ys):
            profiles[ix].append(y[msk].mean())
            pass
        pass

    # Ensure axes exist
    if ax is None:
        _, ax = plt.subplots(figsize=(6,5))
        pass

    # Plot profile(s)
    for ix in range(N):
        ax.hist(profiles[ix], bins=100, label=labels[ix])
        pass

    # Decorations
    ax.set_xlabel('Mass [GeV]')
    ax.set_ylabel('Average Value')
    ax.set_ylim((0,1))
    ax.set_xlim(bins[0], bins[-1])
    ax.legend()

    return ax